## Import modules


In [ ]:
from spezi_data_pipeline.data_access.firebase_fhir_data_access import FirebaseFHIRAccess
from spezi_data_pipeline.data_flattening.fhir_resources_flattener import flatten_fhir_resources, FHIRDataFrame
from spezi_data_pipeline.data_processing.data_processor import FHIRDataProcessor
from spezi_data_pipeline.data_processing.observation_processor import calculate_activity_index
from spezi_data_pipeline.data_exploration.data_explorer import DataExplorer, visualizer_factory, explore_total_records_number
from spezi_data_pipeline.data_export.data_exporter import DataExporter

In [ ]:
from fhirclient import client
from fhirclient.models.questionnaire import Questionnaire
from fhirclient.models.questionnaireresponse import QuestionnaireResponse

# Initialize FHIR client
settings = {
    'app_id': 'my_app',
    'api_base': 'https://launch.smarthealthit.org/v/r4/fhir'
}
smart = client.FHIRClient(settings=settings)

def list_questionnaire_ids():
    """Fetch and list all Questionnaire IDs from the FHIR server."""
    search = Questionnaire.where({})
    questionnaires = search.perform_resources(smart.server)
    
    print("Questionnaire IDs:")
    for questionnaire in questionnaires:
        print(f"ID: {questionnaire.id}, Title: {questionnaire.title}")
        
def get_questionnaire(questionnaire_id):
    """Fetch the Questionnaire resource by ID."""
    return Questionnaire.read(questionnaire_id, smart.server)

def get_questionnaire_response(response_id):
    """Fetch the QuestionnaireResponse resource by ID."""
    return QuestionnaireResponse.read(response_id, smart.server)

def create_question_text_map(questionnaire):
    """Create a mapping of linkId to question text from the Questionnaire."""
    question_text_map = {}
    
    def add_items(items):
        for item in items:
            question_text_map[item.linkId] = item.text
            if item.item:
                add_items(item.item)
    
    add_items(questionnaire.item)
    return question_text_map

def map_response_to_text(response, question_text_map):
    """Map the QuestionnaireResponse linkIds to their corresponding text."""
    def map_items(items):
        for item in items:
            question_text = question_text_map.get(item.linkId, "Unknown question")
            print(f"Question: {question_text}")
            for answer in item.answer:
                # Assuming answers are simple types like boolean, string, etc.
                if hasattr(answer, 'valueBoolean'):
                    print(f"Answer: {answer.valueBoolean}")
                elif hasattr(answer, 'valueString'):
                    print(f"Answer: {answer.valueString}")
                elif hasattr(answer, 'valueInteger'):
                    print(f"Answer: {answer.valueInteger}")
                # Add other types as needed
            
            if item.item:
                map_items(item.item)
    
    map_items(response.item)


In [ ]:
list_questionnaire_ids()

In [ ]:
# Example IDs (replace with actual IDs from your FHIR server)
questionnaire_id = "example-questionnaire"
response_id = "example-response"

# Fetch the Questionnaire and QuestionnaireResponse resources
questionnaire = get_questionnaire(questionnaire_id)
response = get_questionnaire_response(response_id)

# Create the question text map
question_text_map = create_question_text_map(questionnaire)

# Map the response to text and print
map_response_to_text(response, question_text_map)

## Define credential files

In [ ]:
# Define your Firebase project ID
project_id = "spezi-data-pipeline"  # Replace with your Firebase project ID
project_id = "stanfordspezitemplateapp"

# Define the service account key file
service_account_key_file = "path_to_service_account_key_file.json"  # Replace with your service account key file
service_account_key_file = "stanfordspezitemplateapp-firebase-adminsdk-thsmt-ff5e56378d.json"

# Define the collection name where your FHIR observations are stored and the input code if filtering is needed
collection_name = "users"
subcollection_name = "QuestionnaireResponse"

# Define the survey path(s) for creating survey mappings
survey_path = ["SocialSupportQuestionnaire.json"]                

## Initialize the FirebaseFHIRAccess class using your Firebase credentials

In [ ]:
# Initialize the FirebaseFHIRAccess class using your Firebase credentials
firebase_access = FirebaseFHIRAccess(project_id, service_account_key_file)
firebase_access.connect()

fhir_observations = firebase_access.fetch_data(collection_name, subcollection_name)

flattened_fhir_dataframe = flatten_fhir_resources(fhir_observations, survey_path)

In [ ]:
fhir_observations[0].id


In [ ]:
from spezi_data_pipeline.data_flattening.fhir_resources_flattener import extract_mappings

question_mapping, answer_mapping = extract_mappings(survey_path)

# question_mapping

In [ ]:
flattened_fhir_dataframe.df.head()

## Calculate risk score

## Explore Data

## Export data

In [ ]:
exporter = DataExporter(flattened_fhir_dataframe)
exporter.export_to_csv("survey_data.csv") 